In [13]:
import pickle

with open("uniref50_gpt_data.pkl", "rb") as f:
    data = pickle.load(f)
x = data

In [7]:
x = dict(list(data.items())[:10])

In [14]:
from tqdm import tqdm
new = {}
for k, v in tqdm(x.items()):
    _new = {}
    for _k, _v in v.items():
        if _k == "qa":
            _new[_k] = _v[:-1]
        elif _k == "metrics":
            _new[_k] = {i: j for i, j in _v.items() if j != "none" and i != "kingdom"}
        else:
            _new[_k] = _v
    new[k] = _new
with open("data_dict.pkl", "wb") as f:
    pickle.dump(new, f)

100%|██████████| 257167/257167 [00:02<00:00, 118882.40it/s]


In [16]:
next(iter(new.items()))

('A0A009IHW8',
 {'uniref_id': 'UniRef50_A0A009IHW8',
  'sequence': 'MSLEQKKGADIISKILQIQNSIGKTTSPSTLKTKLSEISRKEQENARIQSKLSDLQKKKIDIDNKLLKEKQNLIKEEILERKKLEVLTKKQQKDEIEHQKKLKREIDAIKASTQYITDVSISSYNNTIPETEPEYDLFISHASEDKEDFVRPLAETLQQLGVNVWYDEFTLKVGDSLRQKIDSGLRNSKYGTVVLSTDFIKKDWTNYELDGLVAREMNGHKMILPIWHKITKNDVLDYSPNLADKVALNTSVNSIEEIAHQLADVILNR',
  'fields': ["catalytic activity: NAD(+) = 2'cADPR + H(+) + nicotinamide, H2O + NAD(+) = ADP-D-ribose + H(+) + nicotinamide, EC = 3.2.2.6, H2O + NADP(+) = ADP-D-ribose 2'-phosphate + H(+) + nicotinamide",
   'subunit: In the presence of NAD(+) analog 8-amino-isoquinoline adenine dinucleotide (3AD) forms filaments with 3AD between monomers; conformational changes occur upon 3AD binding., Homodimer',
   'functional domains: NAD+ nucleosidase activity, Toll/interleukin-1 receptor homology (TIR) domain, NAD(P)+ nucleosidase activity, Toll/Interleukin receptor TIR domain, NAD+ nucleotidase, cyclic ADP-ribose generating',
   'taxonomy: Bacteria, Pseudomonado

In [41]:
from pika.utils.data_utils import random_split_df
import pandas as pd

metadata = pd.DataFrame(new.keys(), columns=["uniprot_id"])

metadata.loc[:, "protein_length"] = metadata["uniprot_id"].apply(lambda x: len(new[x]["sequence"]))
metadata.loc[:, "uniref_id"] = metadata["uniprot_id"].apply(lambda x: new[x]["uniref_id"])

metadata.reset_index(drop=True, inplace=True)
import numpy as np

cvd = pd.read_csv("uniprotkb_taxonomy_id_11118_AND_reviewe_2024_02_16.tsv", sep="\t")[['Entry']]

np.random.seed(1)
random_split_df(metadata, [0.945, 0.005, 0.05], key="uniref_id")

cvm = metadata[metadata.uniprot_id.isin(cvd["Entry"].to_list())]

metadata["split"] = metadata.apply(
    lambda row: "test" if row["uniref_id"] in cvm["uniref_id"].to_list() else row["split"], axis=1
)

metadata

,uniprot_id,protein_length,uniref_id,split
0,A0A009IHW8,269,UniRef50_A0A009IHW8,train
1,A0A067XGX8,512,UniRef50_A0A067XH53,train
2,A0A067XH53,533,UniRef50_A0A067XH53,train
3,A0A068BGA5,456,UniRef50_A0A068BGA5,train
4,A0A072VHJ1,481,UniRef50_A0A072VHJ1,train
...,...,...,...,...
257162,Q9ZWB3,471,UniRef50_Q9ZWB3,train
257163,S3DQP8,358,UniRef50_S3DQP8,train
257164,V6F510,297,UniRef50_V6F510,test
257165,W6KHH6,293,UniRef50_W6KHH6,train


In [42]:
metadata[metadata.uniprot_id.isin(cvd["Entry"].to_list())]["split"].value_counts()

split
test    147
Name: count, dtype: int64

In [43]:
metadata.to_csv("s1.csv")

In [28]:
cvd

,Entry
0,A3EX94
1,A3EXD0
2,A3EXG6
3,K9N4V7
4,K9N5Q8
...,...
481,Q9QAQ7
482,Q9QAR4
483,Q9QAS0
484,Q9QAS1
